In [1]:
import numpy as np
import xarray as xr

from tools.landsat_utils import easy_raster, normalize_band as normalize
from tools.utils import create_polydict_ptsarr, xy2ll, directory_spider
from tools.dem_utils import sample_along_profile
from tools.flexural_fitting import fitting_REMA_dh

import matplotlib.pyplot as plt
import cmcrameri.cm as cmc
from tools.plot_tools import plot_latlon_lines, shifted_colormap
from matplotlib import gridspec
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar

In [2]:
import plotSettings
COLS = plotSettings.COLS
plotSettings.paper()

In [3]:
%matplotlib tk

In [4]:
# general data directory
data_dir = '/'

In [5]:
collapse_dirs = ['data/LandSat8-9_Color/LC08_L1GT_125111_20150329_20200909_02_T2',                  
                 'data/LandSat8-9_Color/LC08_L1GT_127111_20161210_20201016_02_T2'
                ]
REMA_files = ['data/REMA/2014/SETSM_BBMASKED_s2s041_WV02_20140302_103001002EB86900_103001002D317400_2m_lsf_seg1_dem.tif',
              'data/REMA/2016/SETSM_BBMASKED_s2s041_WV01_20161206_102001005547E000_1020010059AF2400_2m_lsf_seg1_dem.tif',
              'data/REMA/2019/SETSM_BBMASKED_s2s041_WV01_20191119_102001008C642200_102001008F9B8C00_2m_lsf_seg1_dem.tif',
              'data/REMA/2021/SETSM_BBMASKED_s2s041_WV03_20211010_104001006C7FF100_104001006E0B1400_2m_lsf_seg1_dem.tif'
             ]

LagrangianDiff_files = ['data/REMA/LagrangianDifferencing/lagrangian_demdiff_2016-2014.nc',
                        'data/REMA/LagrangianDifferencing/lagrangian_demdiff_2021-2014.nc'
                       ]

masked_IS2 = directory_spider(data_dir, path_pattern='ICESat', file_pattern='.pkl')
masked_IS2 = [file for file in masked_IS2 if '._' not in file]
TRKS = ['0401', '0470', '0843', '0']
is2_files = []
is2_files.append([file for file in masked_IS2 if f'/2019/' in file and f'/0401/' in file][1])
is2_files.append([file for file in masked_IS2 if f'/2019/' in file and f'/0470/' in file][0])
is2_files.append([file for file in masked_IS2 if f'/2018/' in file and f'/0843/' in file][0])

# bounding box locations for context image
bb_filename = os.path.join(data_dir, 'GIS/BoundingBoxes2.shp')
print('loading mask')
bbdict, bbpts = create_polydict_ptsarr(bb_filename)
mask = bbdict[0]

loading mask


In [6]:
SAVE = False
fig_save_dir = 'figs'
fig_type = '.svg'
save_name = 'figure2'

left, bottom, right, top = mask.bounds
# max and minimum uplift
dh_min, dh_max = -20, 7
# colormap for elevation differences, "centered" at dh = 0
scmap = shifted_colormap(cmc.vik, midpoint=1 - (dh_max) / (dh_max + abs(dh_min)))
# colormap for elevations
elev_cmap = cmc.batlowW

#############################################################################################################
# set up figure
#############################################################################################################
plt.close('all')
fig = plt.figure(num=1,clear=1,
                 figsize=[14.53866667,  9.52533333])

PARENT = gridspec.GridSpec(3,2*2, figure=fig, height_ratios=[1.5,1,1])

gs0 = gridspec.GridSpecFromSubplotSpec(1,3, subplot_spec=PARENT[0,:], wspace=0.03)
imaxs = [fig.add_subplot(gs0[i]) for i in range(3)]

gs1 = gridspec.GridSpecFromSubplotSpec(2,2, subplot_spec=PARENT[1:,:],
                                       hspace=0.01, wspace=0.01)
REMA_profs = [fig.add_subplot(gs1[0,0]), fig.add_subplot(gs1[0,1])]
IS2_profs = fig.add_subplot(gs1[1,1])

#############################################################################################################
# context images
#############################################################################################################
ll=xy2ll(left, bottom), xy2ll(left, top), xy2ll(right, top), xy2ll(right, bottom)
ll=np.array(ll)
lon_range = [ll[:,0].min(), ll[:,0].max()]
lat_range = [ll[:,1].min(), ll[:,1].max()]

for i, ax in enumerate(imaxs[:2]):
    red,green,blue='B4','B3','B2'
    bands=['B2','B3','B4','B8']
    rast = easy_raster(os.path.join(data_dir, collapse_dirs[i]),
                       bands=bands,
                       mask=mask,
                       always_TOA=True)
    ax.imshow(rast.masked['B8'], cmap=plt.cm.gray, extent=(left, right, bottom, top), rasterized=True)
    ax = plot_latlon_lines(ax, lat_range=lat_range, lon_range=lon_range,
                            extents=(left, right, bottom, top),
                            lon_text=True if i == 0 else False)
    ax.set(ylim=(bottom, top), xlim=(left, right),
           yticklabels=[], xticklabels=[],
           xticks=[], yticks=[])
    
    scalebar = AnchoredSizeBar(ax.transData,
                               1e3, '1 km', 'lower right', 
                               pad=1,
                               frameon=False,
                               size_vertical=100,
                              )

    ax.add_artist(scalebar)
    [a.set_linewidth(1) for _,a in ax.spines.items()]

#############################################################################################################
# DEM/DEM difference overlays
#############################################################################################################
ds = xr.open_dataset(os.path.join(data_dir, LagrangianDiff_files[0]))

p = imaxs[-1].imshow(ds.dh, 
                 extent=(left, right, bottom, top),
                 vmin=-20, vmax=7, cmap=scmap)
imaxs[-1].set(ylim=(bottom, top), xlim=(left, right),
              yticklabels=[], xticklabels=[],
              xticks=[], yticks=[])
imaxs[-1] = plot_latlon_lines(imaxs[-1], lat_range=lat_range, lon_range=lon_range,
                              extents=(left, right, bottom, top),
                              lon_text=False)

inset = imaxs[-1].inset_axes([0.1, 0.166, 1-0.433, 0.07])
inset.xaxis.set_ticks_position('bottom')
cb = fig.colorbar(p, cax=inset, 
                  label='Elevation difference $\Delta h$ (m)',
                  orientation='horizontal', extend='both',
                 )
cb.set_ticks([dh_min, 0, dh_max])
[a.set_linewidth(1) for _,a in imaxs[-1].spines.items()]


dem1 = ds.dem1_lagrangian.values.copy()
s = np.nanstd(dem1)
dem1[dem1 < 80] = np.nan
dem1[dem1 > 140] = np.nan

p = imaxs[0].contourf(ds.x[::50], ds.y[::50], dem1[::50, ::50],
                        vmin=84, vmax=132, cmap=cmc.lapaz, alpha=0.5)
inset = imaxs[0].inset_axes([0.1, 0.166, 1-0.433, 0.07])#1-0.433, 0.07])
inset.xaxis.set_ticks_position('bottom')
cb = fig.colorbar(p, cax=inset, 
                  label='Surface elevation (m)',
                  orientation='horizontal',
                 )
cbticks = ['{0}'.format(f'{v:0.0f}' if i%2==0 else '') for i, v in enumerate(cb.get_ticks())]
cb.set_ticklabels(cbticks)

dem2 = ds.dem2.values.copy()
s = np.nanstd(dem2)
dem2[dem2 < 80] = np.nan
dem2[dem2 > 140] = np.nan

p = imaxs[1].contourf(ds.x[::50], ds.y[::50], dem2[::50, ::50],
                        vmin=84, vmax=132, cmap=cmc.lapaz, alpha=0.5)

inset = imaxs[1].inset_axes([0.1, 0.166, 1-0.433, 0.07])#1-0.433, 0.07])
inset.xaxis.set_ticks_position('bottom')
cb = fig.colorbar(p, cax=inset, 
                  label='Surface elevation (m)',
                  orientation='horizontal',
                 )

cbticks = ['{0}'.format(f'{v:0.0f}' if i%2==0 else '') for i, v in enumerate(cb.get_ticks())]
cb.set_ticklabels(cbticks)

#############################################################################################################
# DEM/DEM difference overlays
#############################################################################################################
for i, f in enumerate(is2_files):
    ans = fitting_REMA_dh(f, os.path.join(data_dir, LagrangianDiff_files[0]),
                          gt='gt3' if ('/0843/' in f or '/0028/' in f) else 'gt1',
                          
                         )
    x_is2, y_is2, lat, lon, h_is2, r, dh_rema, uplift, surface, C_C = ans
    IS2_profs.plot(lat, h_is2, c=COLS[i])
    
    h = sample_along_profile(ds, 'dem1_lagrangian', x_is2, y_is2)
    REMA_profs[0].plot(lat, h, c=COLS[i])
    
    h = sample_along_profile(ds, 'dem2', x_is2, y_is2)
    REMA_profs[1].plot(lat, h, c=COLS[i])
    
    [ax.plot(x_is2, y_is2, c=COLS[i]) for ax in imaxs]
    
h_ylims = [np.min([REMA_profs[0].get_ylim()[0], REMA_profs[1].get_ylim()[0], IS2_profs.get_ylim()[0]]),
           np.max([REMA_profs[0].get_ylim()[1], REMA_profs[1].get_ylim()[1], IS2_profs.get_ylim()[0]])
          ]
yticks = [a for a in cb.get_ticks() if (a >= h_ylims[0] and a <= h_ylims[1])]
xticks = [-72.05 - 0.025, -72.05, -72.025, -72.0, -72 + 0.025]
xticklabels = ['{0}°'.format(f'{v:0.2f}')  for i,v in enumerate(xticks)]

for ax in [REMA_profs[0], REMA_profs[1], IS2_profs]:
    ax.set_xlim(lat_range)
    ax.set_ylim(h_ylims)   
    ax.set_xticks(xticks)
    ax.set_xticklabels(xticklabels)
    [ax.axvline(v, c='k', ls='--', lw=0.3) for v in [-72.05, -72.0]]
    ax.set_yticks(yticks)

    
REMA_profs[1].set(xticklabels=[], yticklabels=[])
REMA_profs[0].set(xlabel='Latitiude', ylabel='Surface elevation (m)')
IS2_profs.set(xlabel='Latitude', ylabel='Surface elevation (m)')

if SAVE:
    plt.savefig(os.path.join(fig_save_dir, save_name + fig_type), dpi=600)

['LC08_L1GT_125111_20150329_20200909_02_T2_B2.TIF', '._LC08_L1GT_125111_20150329_20200909_02_T2_B2.TIF']
['LC08_L1GT_127111_20161210_20201016_02_T2_B2.TIF', '._LC08_L1GT_127111_20161210_20201016_02_T2_B2.TIF']
260 35
239 42
262 45


[90.0, 96.0, 102.0, 108.0, 114.0]

In [23]:
h_ylims[1]

117.70625

In [10]:
xticklabels

['<generator object <genexpr> at 0x2a409d690>']

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
for f in is2_files:
    ans = fitting_REMA_dh(f, os.path.join(data_dir, LagrangianDiff_files[0]),
                          gt='gt3' if ('/0843/' in f or '/0028/' in f) else 'gt1',
                          beam='L'
                         )
    x_is2, y_is2, lat, lon, h_is2, r, dh_rema, uplift, surface, C_C = ans
    ax.plot(lon, lat, h_is2)
    

ax.set(xlim=lon_range, ylim=lat_range, zlim=h_ylims,
       yticks=[lat_range[0], -72.05, -72, lat_range[1]], 
       yticklabels=['{0:0.02f}'.format(s) for s in [lat_range[0], -72.05, -72, lat_range[1]]],
       xticks=[lon_range[0], 69.45, 69.6, lon_range[1]], 
       xticklabels=['{0:0.02f}'.format(s) for s in [lon_range[0], 69.45, 69.6, lon_range[1]]]
      )
ax.grid(which='major', axis='x', c='k', ls='--', lw=0.3, alpha=1)
# make the panes transparent
ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0))
# make the grid lines transparent
# ax.xaxis._axinfo["grid"]['color'] =  (1,1,1,0)
# ax.yaxis._axinfo["grid"]['color'] =  (1,1,1,0)
# ax.zaxis._axinfo["grid"]['color'] =  (1,1,1,0)
ax.set_aspect('equal')

In [ ]:
plt.figure()
plt.contourf(ds.x[::50], ds.y[::50], dem1[::50, ::50],
             vmin=84, vmax=132, cmap=cmc.buda, alpha=0.5,
            )#linewidths=10)

In [ ]:
plt.plot(lat, h)
plt.gca().set_xticks([lat_range[0], -72.05, -72, lat_range[1]])
plt.gca().set_xticklabels(['{0:0.02f}'.format(s) for s in [lat_range[0], -72.05, -72, lat_range[1]]]) 
plt.grid(which='major', axis='x', c='k', ls='--', lw=0.3, alpha=1)

In [ ]:
plt.contour(ds.x, ds.y, ds.dem1_lagrangian, levels=np.arange(80, 140, 5))
plt.colorbar()

In [ ]:
dem1 = ds.dem1_lagrangian.values.copy()
s = np.nanstd(dem1)
dem1[dem1 < 80] = np.nan
dem1[dem1 > 140] = np.nan

plt.contour(ds.x, ds.y, dem1, levels=np.arange(80, 140, 5))
plt.colorbar()

In [ ]:
from scipy import interpolate

In [ ]:
# FUN = interpolate.interp2d(ds.x, ds.y, dem1)
FUN = interpolate.RectBivariateSpline(np.flip(ds.y.values), ds.x, dem1)
# i
# FUN = interpolate.RegularGridInterpolator((ds.y, ds.x), dem1, 
#                                           method='slinear', bounds_error=False, fill_value=np.nan)

In [ ]:
np.flip(np.array(ds.y.values))

In [ ]:
X,Y = np.linspace(np.min(ds.x), np.max(ds.x), 100), np.linspace(np.min(ds.y), np.max(ds.y), 100)
X,Y = np.meshgrid(X,Y)
PTS = np.array(list(zip(Y.flatten(), X.flatten())))
plt.contour(X,Y, FUN(Y,X), 
            levels=np.arange(80, 140, 3), lw=0.1, cmap='gist_earth')

In [ ]:
FUN(PTS)

In [ ]:
plt.contour(ds.x[::50], ds.y[::50], dem1[::50, ::50],
            levels=np.arange(80, 140, 2), lw=0.1, cmap='gist_earth')